In [13]:
import os
from PIL import Image
import glob
import cv2
import numpy as np

def center_crop_image(image_path, output_folder, crop_size=(128, 128)):
    # 打开图像
    img = Image.open(image_path)
    width, height = img.size

    # 计算中心裁剪的区域
    left = (width - crop_size[0]) / 2
    top = (height - crop_size[1]) / 2
    right = (width + crop_size[0]) / 2
    bottom = (height + crop_size[1]) / 2

    # 裁剪图像
    cropped_img = img.crop((left, top, right, bottom))

    # 转换为NumPy数组
    image_np = np.array(cropped_img)

    # 检查图像是否是灰度图像，如果是灰度图像，则不进行颜色转换
    if len(image_np.shape) == 3:
        # 使用OpenCV进行中值滤波
        denoised_image_np = cv2.medianBlur(image_np, 5)  # 参数5表示滤波窗口大小
    else:
        # 如果是灰度图像，直接应用滤波
        denoised_image_np = cv2.medianBlur(image_np, 5)

    cropped_img = Image.fromarray(denoised_image_np)

    # 获取图像文件名（不带扩展名）
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 保存裁剪后的图像
    cropped_img.save(os.path.join(output_folder, f"{image_name}_center_cropped.png"))

def process_folder(input_folder, output_base_folder, max_images_per_subfolder=10):
    # 遍历输入文件夹中的所有子文件夹
    for subdir, _, files in os.walk(input_folder):
        # 初始化计数器
        image_count = 0

        # 遍历子文件夹中的文件
        for file in files:
            # 只处理图像文件
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
                image_path = os.path.join(subdir, file)

                # 获取相对路径以保持子文件夹结构
                relative_path = os.path.relpath(subdir, input_folder)
                output_folder = os.path.join(output_base_folder)

                # 裁剪图像并保存
                center_crop_image(image_path, output_folder)

                # 增加计数器
                image_count += 1

                # 如果达到最大数量，跳过该子文件夹的剩余图像
                if image_count >= max_images_per_subfolder:
                    break

In [14]:
input_folder = "../load/benchmark/Google-15-real"  # 替换为你的输入文件夹路径
output_base_folder = "../load/benchmark/Google-15-split-to-patch"  # 替换为你的输出文件夹路径
process_folder(input_folder, output_base_folder)
# glob.glob(os.path.join(input_folder, "**", "*.*"), recursive=True)